In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
plt.style.use('./my_style.py')

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

def interp_df_fun(col, k_in, k_out, deg):
    return InterpolatedUnivariateSpline(k_in, col, k=deg)(k_out)

def interp_df(df, N_points=1000, deg=3, mode='log'):
    k_min = 1e-3
    k_max = 1
    if mode=='lin':
        k_out = np.linspace(k_min, k_max, N_points)
    elif mode=='log':
        k_out = np.logspace(np.log10(k_min), np.log10(k_max), N_points)
    df = df.apply(lambda col: interp_df_fun(col, df.index, k_out, deg)) 
    df.index = k_out
    return df

In [ ]:
files = glob.glob('./Data/Output/n*-beta0p2-k1/out_table.txt')

In [ ]:
files

In [ ]:
header = 'a_E a_J E_J dE_J/da phi_J dphi_J/da G_eff/G A_conf D_kmou'.split(' ')

In [ ]:
def read_out_table(f):
    dum_df = pd.read_csv(f, skiprows=100, sep='\t', names=header).set_index('a_E')
    return interp_df(dum_df)

In [ ]:
df_n2_old = read_out_table('./Data/Output/n2-beta0p2-k1/out_table_old.txt')

In [ ]:
df = pd.concat([read_out_table(f) for f in files], 
               axis=1, 
               keys=[f.split('/')[3].split('-')[0] for f in files]).sort_index(axis=1)

In [ ]:
idx = pd.IndexSlice

In [ ]:
import matplotlib.ticker as mtick

In [ ]:
ax = df.apply(lambda x: x/df['n2', x.name[1]].values-1).loc[:, idx[:,'D_kmou']].plot(logx=True, 
                                                                              xlabel=r'$a_E$',
                                                                             ylabel=r'$\Delta D_{\rm case}/D_{\rm n2}$',
                                                                             )
(df_n2_old/df['n2']-1)['D_kmou'].plot(ax=ax)
ax.yaxis.set_major_formatter(mtick.FuncFormatter(lambda y, _: '{:.2%}'.format(y))) 
plt.legend(['n2','n3','n4','n5', 'n2 Ben'])
plt.savefig('./Figures/ngt2_growth_comparison.pdf', format='pdf')